## Image Classification MNIST Dataset With CNN 1 Convolution, and Jittering, Normalization Images

## Import Library

In [1]:
import torch
import torch.optim as optim
from torch import nn
from torchvision import datasets,transforms
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

## Jittering and Normalization Image Dataset

In [2]:
set_latih = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,)),transforms.ColorJitter()]), 
    download = True,            
)
set_uji = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,)),transforms.ColorJitter()]),
    download = True,
)
latih = torch.utils.data.DataLoader(set_latih, batch_size=64, shuffle=True)
uji = torch.utils.data.DataLoader(set_uji, batch_size=64, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 76051831.88it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 112058921.21it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21780002.76it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20202045.35it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Set learning rate, momentum, and make device with GPU

In [3]:
learning_rate = 0.01
momentum = 0.5
device = ("cuda" if torch.cuda.is_available() else "cpu")

## Make Training Model with 1 CNN and 1 Linear Layer

In [4]:
class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.convolutaional_neural_network_layers = nn.Sequential(
                nn.Conv2d(in_channels=1, out_channels=5, kernel_size=5, padding=1, stride=1), 
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
        )
        self.linear_layers = nn.Sequential(
                nn.Linear(in_features=5*13*13, out_features=20),          
                nn.ReLU(),
                nn.BatchNorm1d(num_features=20),
                nn.Dropout(p=0.2),
                nn.Linear(in_features=20, out_features=10)
        )

    def forward(self, x):
        x = self.convolutaional_neural_network_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

## Make Model from Class Network and Then Using SGD Optimizer and Cross Entropy Loss

In [5]:
model = Network()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)
criteria = nn.CrossEntropyLoss()
model.to(device)

Network(
  (convolutaional_neural_network_layers): Sequential(
    (0): Conv2d(1, 5, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=845, out_features=20, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=20, out_features=10, bias=True)
  )
)

## Process Training and Testing From Model

In [6]:
train_loss, val_loss = [], []
accuracy_total_train, accuracy_total_val = [], []
max_epoch = 10
for epoch in range(max_epoch):
   
    total_train_loss = 0
    total_val_loss = 0
    model.train()
    total = 0

    for idx, (image, label) in enumerate(latih):
        gambar, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        pred = model(gambar)
        loss = criteria(pred, label)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
                
    accuracy_train = total / len(set_latih)
    accuracy_total_train.append(accuracy_train)
    total_train_loss = total_train_loss / (idx + 1)
    train_loss.append(total_train_loss)
    model.eval()
    total = 0

    for idx, (image, label) in enumerate(uji):
        gambar, label = image.cuda(), label.cuda()
        pred = model(gambar)
        loss = criteria(pred, label)
        total_val_loss += loss.item()
        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1

    accuracy_val = total / len(set_uji)
    accuracy_total_val.append(accuracy_val)
    total_val_loss = total_val_loss / (idx + 1)
    val_loss.append(total_val_loss)

    if epoch % 2 == 0:
      print("Epoch: {}  ".format(epoch),
            "Training loss: {:.2f}  ".format(total_train_loss),
            "Testing loss: {:.2f}  ".format(total_val_loss),
            "Train accuracy: {:.2f}  ".format(accuracy_train),
            "Test accuracy: {:.2f}  ".format(accuracy_val))

Epoch: 0   Training loss: 0.44   Testing loss: 0.14   Train accuracy: 0.90   Test accuracy: 0.97  
Epoch: 2   Training loss: 0.13   Testing loss: 0.07   Train accuracy: 0.97   Test accuracy: 0.98  
Epoch: 4   Training loss: 0.10   Testing loss: 0.07   Train accuracy: 0.97   Test accuracy: 0.98  
Epoch: 6   Training loss: 0.09   Testing loss: 0.06   Train accuracy: 0.97   Test accuracy: 0.98  
Epoch: 8   Training loss: 0.08   Testing loss: 0.05   Train accuracy: 0.98   Test accuracy: 0.98  
